In [5]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmatize and lower each word and remove duplicates
words = [WordNetLemmatizer().lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print(len(documents), "documents")
# classes = intents
print(len(classes), "classes", classes)
# words = all words, vocabulary
print(len(words), "unique lemmatized words", words)

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# create our training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in an attempt to represent related words
    pattern_words = [WordNetLemmatizer().lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in the current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for the current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append({'input': bag, 'output': output_row})

# shuffle our features and turn into np.array
random.shuffle(training)
training_data = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = np.array([item['input'] for item in training_data])
train_y = np.array([item['output'] for item in training_data])
print("Training data created")

# Create model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
from keras.optimizers import schedules

# Define a learning rate schedule
lr_schedule = schedules.ExponentialDecay(initial_learning_rate=0.01, decay_steps=100000, decay_rate=0.96, staircase=True)

# Use the learning rate schedule in the optimizer
sgd = SGD(learning_rate=lr_schedule, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
#model.save('chatbot_model.h5', hist)
model.save('chatbot_model.keras', hist)

print("Model created")


84 documents
29 classes ['amenities', 'beach_access', 'booking', 'breakfast', 'business', 'check_in', 'check_out', 'cleaning', 'complaints', 'fitness_center', 'goodbye', 'greeting', 'late_checkout', 'laundry_services', 'local_attractions', 'location', 'lost_and_found', 'parking', 'pets', 'pool', 'reservations', 'restaurant', 'room_service', 'room_types', 'spa', 'special_occasions', 'thankyou', 'transportation', 'weddings']
155 unique lemmatized words ["'s", ',', 'a', 'about', 'access', 'additional', 'address', 'airport', 'allowed', 'amenity', 'an', 'and', 'anniversary', 'any', 'appreciate', 'are', 'arrange', 'at', 'attraction', 'available', 'beach', 'birthday', 'book', 'breakfast', 'bring', 'business', 'bye', 'can', 'cancel', 'cancellation', 'car', 'care', 'cat', 'center', 'check', 'check-in', 'check-out', 'checkout', 'choice', 'cleaning', 'complaint', 'conference', 'coordinator', 'cost', 'deal', 'direction', 'distance', 'do', 'document', 'doe', 'dog', 'early', 'equipment', 'fax', 'fea

17/17 [==============================] - 0s 2ms/step - loss: 0.2720 - accuracy: 0.9167
Epoch 64/200
17/17 [==============================] - 0s 2ms/step - loss: 0.4498 - accuracy: 0.8810
Epoch 65/200
17/17 [==============================] - 0s 2ms/step - loss: 0.3300 - accuracy: 0.8571
Epoch 66/200
17/17 [==============================] - 0s 2ms/step - loss: 0.2958 - accuracy: 0.8929
Epoch 67/200
17/17 [==============================] - 0s 2ms/step - loss: 0.2531 - accuracy: 0.9167
Epoch 68/200
17/17 [==============================] - 0s 2ms/step - loss: 0.3217 - accuracy: 0.9286
Epoch 69/200
17/17 [==============================] - 0s 2ms/step - loss: 0.3788 - accuracy: 0.8690
Epoch 70/200
17/17 [==============================] - 0s 2ms/step - loss: 0.2999 - accuracy: 0.8929
Epoch 71/200
17/17 [==============================] - 0s 2ms/step - loss: 0.2019 - accuracy: 0.9524
Epoch 72/200
17/17 [==============================] - 0s 2ms/step - loss: 0.2943 - accuracy: 0.9286
Epoch 73/200


17/17 [==============================] - 0s 4ms/step - loss: 0.1798 - accuracy: 0.9286
Epoch 145/200
17/17 [==============================] - 0s 4ms/step - loss: 0.2211 - accuracy: 0.9524
Epoch 146/200
17/17 [==============================] - 0s 4ms/step - loss: 0.1355 - accuracy: 0.9762
Epoch 147/200
17/17 [==============================] - 0s 4ms/step - loss: 0.2131 - accuracy: 0.9286
Epoch 148/200
17/17 [==============================] - 0s 4ms/step - loss: 0.1729 - accuracy: 0.9643
Epoch 149/200
17/17 [==============================] - 0s 4ms/step - loss: 0.1330 - accuracy: 0.9524
Epoch 150/200
17/17 [==============================] - 0s 4ms/step - loss: 0.1959 - accuracy: 0.9405
Epoch 151/200
17/17 [==============================] - 0s 4ms/step - loss: 0.1089 - accuracy: 0.9643
Epoch 152/200
17/17 [==============================] - 0s 4ms/step - loss: 0.2755 - accuracy: 0.9286
Epoch 153/200
17/17 [==============================] - 0s 4ms/step - loss: 0.1113 - accuracy: 0.9762
Epoc

In [7]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
import json
import random

import tkinter
from tkinter import *

model = load_model('chatbot_model.keras')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))


def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

'''def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res'''

def chatbot_response(msg):
    ints = predict_class(msg, model)
    if not ints:
        # No intent was predicted, provide a default response
        return "I'm sorry, I didn't understand that."
    
    res = getResponse(ints, intents)
    return res

#Creating GUI with tkinter
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 

base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

1/1 [==============================] - 0s 15ms/step
